# Delete Redshift Notebook

In [1]:
import boto3
import time
import pandas as pd
import os

In [2]:
# move up one directory for config files
os.chdir('..')
os.getcwd()

'/home/workspace'

In [3]:
# load configuration file

from configparser import ConfigParser

config = ConfigParser()
config.read_file(open('dwh.cfg'))

KEY=config.get('AWS','key')
SECRET= config.get('AWS','secret')
DB_HOST = config.get("CLUSTER","HOST")
DB_ROLE_NAME = config.get("CLUSTER", "DB_ROLE_NAME")


In [4]:
# Create Redshift client
from boto3 import client

redshift = client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET)

In [5]:
# Create IAM Client

from boto3 import client

iam = client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name='us-west-2'
                  )

In [6]:
# delete Redshift resources
try:
    redshift.delete_cluster( ClusterIdentifier=DB_HOST,  SkipFinalClusterSnapshot=True)
except Exception as e:
    print(e)

In [7]:
# verify deletion

def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])



starttime=time.time()
while (redshift.describe_clusters(ClusterIdentifier=DB_HOST)['Clusters'][0] == 'deleting') == True:
    print("Checking every minute for redshift deletion...")
    time.sleep(60.0 - ((time.time() - starttime) % 60.0))

try:    
    myClusterProps = redshift.describe_clusters(ClusterIdentifier=DB_HOST)['Clusters'][0]
    prettyRedshiftProps(myClusterProps)
except ClusterNotFoundFault as e:
    print(e)


In [8]:
# verify deletion
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DB_HOST)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

,Key,Value
0,ClusterIdentifier,dwhcluster
1,NodeType,dc2.large
2,ClusterStatus,deleting
3,MasterUsername,dwhuser
4,DBName,dwh
5,Endpoint,"{'Address': 'dwhcluster.cysyqtwog01l.us-west-2.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-4f147f37
7,NumberOfNodes,4


In [9]:
#remove roles and policies
iam.detach_role_policy(RoleName=DB_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=DB_ROLE_NAME)

{'ResponseMetadata': {'RequestId': 'ac7a01be-df3c-4419-bb98-372ceb117b3e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ac7a01be-df3c-4419-bb98-372ceb117b3e',
   'content-type': 'text/xml',
   'content-length': '200',
   'date': 'Tue, 02 Jun 2020 18:19:29 GMT'},
  'RetryAttempts': 0}}